In [ ]:
# !pip install unsloth
# !pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

  Cloning https://github.com/unslothai/unsloth.git to c:\users\ayush\appdata\local\temp\pip-req-build-qfp98kic
  Resolved https://github.com/unslothai/unsloth.git to commit 8a055402a27c3d9643cc16947ce40311a280e69c
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for unsloth: filename=unsloth-2025.4.8-py3-none-any.whl size=265064 sha256=c0815f2647b5284f1bfb6951345efefd9da41086194cd7a1a82a04f0de2854bb
  Stored in directory: C:\Users\ayush\AppData\Local\Temp\pip-ephem-wheel-cache-bt70xzl7\wheels\0b\bf\f5\61523189908a01bce8752a181f02f8b057ffc2c792447d39ff
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.4.3
    Uninstalling unsloth-2025.4.3:
 

  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git 'C:\Users\ayush\AppData\Local\Temp\pip-req-build-qfp98kic'


In [1]:
from unsloth import FastLanguageModel
import torch 
from trl import SFTTrainer 
from unsloth import is_bfloat16_supported 
from huggingface_hub import login 
from transformers import TrainingArguments 
from datasets import load_dataset 
import wandb
from kaggle_secrets import UserSecretsClient

ModuleNotFoundError: No module named 'triton'

In [ ]:
user_secrets = UserSecretsClient() 
hugging_face_token = user_secrets.get_secret("Hugging_Face_Token")
wnb_token = user_secrets.get_secret("wnb")

login(hugging_face_token) 

wandb.login(key=wnb_token) 
run = wandb.init(
    project='', 
    job_type="training", 
    anonymous="allow"
)

In [ ]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",  
    max_seq_length=max_seq_length, 
    dtype=dtype, 
    load_in_4bit=load_in_4bit, 
    token=hugging_face_token, 
)

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


NameError: name 'model' is not defined

In [ ]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and provide a step-by-step chain of reasoning to ensure a comprehensive and scientifically grounded response.

### Instruction:
You are an agricultural expert with deep knowledge in crop science, soil management, sustainable farming practices, agri-tech innovations, and climate-resilient agriculture. 
Please answer the following agriculture-related question.

### Question:
{}

### Response:
{}"""

In [ ]:
question = """How can I improve water efficiency in agriculture?"""

FastLanguageModel.for_inference(model)  

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")  

outputs = model.generate(
    input_ids=inputs.input_ids, 
    attention_mask=inputs.attention_mask, 
    max_new_tokens=1200, 
    use_cache=True, 
)

response = tokenizer.batch_decode(outputs)

print(response[0].split("### Response:")[1])

In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and provide a step-by-step chain of reasoning to ensure a comprehensive and scientifically grounded response.

### Instruction:
You are an agricultural expert with deep knowledge in crop science, soil management, sustainable farming practices, agri-tech innovations, and climate-resilient agriculture. 
Please answer the following agriculture-related question.

### Question:
{}

### Response:
{}"""

In [ ]:
dataset = load_dataset("KisanVaani/agriculture-qa-english-only", split = "train[0:500]",trust_remote_code=True)
dataset

In [ ]:
dataset[1]

In [ ]:
EOS_TOKEN = tokenizer.eos_token  
EOS_TOKEN

In [ ]:
def formatting_prompts_func(examples):  
    inputs = examples["input"]         
    outputs = examples["response"]      

    texts = []  

    for input, output in zip(inputs, outputs):  
        text = train_prompt_style.format(input, output) + EOS_TOKEN  
        texts.append(text)  

    return {
        "text": texts,  
    }

In [ ]:
dataset_finetune = dataset.map(formatting_prompts_func, batched = True)
dataset_finetune["text"][0]

In [ ]:
model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[  
        "q_proj",   
        "k_proj",   
        "v_proj",   
        "o_proj",   
        "gate_proj",  
        "up_proj",    
        "down_proj",  
    ],
    lora_alpha=16,  
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  
    random_state=3407,  
    use_rslora=False,  
    loftq_config=None,  
)

In [ ]:
trainer = SFTTrainer(
    model=model_lora,  
    tokenizer=tokenizer,  
    train_dataset=dataset_finetune,  
    dataset_text_field="text",  
    max_seq_length=max_seq_length,  
    dataset_num_proc=2,  

    args=TrainingArguments(
        per_device_train_batch_size=2,  
        gradient_accumulation_steps=4,  
        num_train_epochs=1, 
        warmup_steps=5,  
        max_steps=60,  
        learning_rate=2e-4,  
        fp16=not is_bfloat16_supported(),  
        bf16=is_bfloat16_supported(),  
        logging_steps=10,  
        optim="adamw_8bit",  
        weight_decay=0.01,  
        lr_scheduler_type="linear",  
        seed=3407,  
        output_dir="outputs",  
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
trainer_stats

In [ ]:
question = """What are some methods for controlling pests and diseases in greenhouse tomato production?"""

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model_lora.generate(
    input_ids=inputs.input_ids,          
    attention_mask=inputs.attention_mask, 
    max_new_tokens=1200,                  
    use_cache=True,                        
)

response = tokenizer.batch_decode(outputs)

print(response[0].split("### Response:")[1])

In [ ]:
from huggingface_hub import HfApi
from peft import PeftModel
import os
repo_name = "NamanSharma15/Agricultural-Finetuned-LLama-3.1-8B"
tokenizer.push_to_hub(repo_name)
model_lora.push_to_hub(repo_name)

print(f"Successfully pushed LoRA model and tokenizer to: https://huggingface.co/{repo_name}")

In [ ]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="NamanSharma15/Agricultural-Finetuned-LLama-3.1-8B",  
    max_seq_length=max_seq_length, 
    dtype=dtype, 
    load_in_4bit=load_in_4bit, 
    token=hugging_face_token, 
)


In [ ]:
question = """How can I improve water efficiency in agriculture?""" 

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")  

outputs = model_lora.generate(
    input_ids=inputs.input_ids, 
    attention_mask=inputs.attention_mask, 
    max_new_tokens=1200, 
    use_cache=True, 
)

response = tokenizer.batch_decode(outputs)

print(response[0].split("### Response:")[1])